# Pandas (2)

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("http://data.insideairbnb.com/spain/comunidad-de-madrid/madrid/2023-03-15/visualisations/listings.csv", index_col="id")
df.head()

# Agrupamiento y ordenamiento de datos

## Agrupamiento

In [ ]:
df["neighbourhood_group"].value_counts()

In [ ]:
#df.groupby("neighbourhood_group").price.count()
#df.groupby("neighbourhood_group").price.max()
#df.groupby("neighbourhood_group").price.min()
df.groupby("neighbourhood_group").price.mean()

In [ ]:
df.groupby("neighbourhood_group").price.agg(["min", "max", "mean", "sum", "count"])

In [ ]:
df.groupby(["neighbourhood_group", "neighbourhood"]).price.agg(["min", "max", "mean", "sum", "count"])

In [ ]:
df.groupby(["neighbourhood_group", "neighbourhood"]).price.agg(["min", "max", "mean", "sum", "count"]).reset_index()

## Ordenamiento

In [ ]:
df.sort_values(by=["host_name", "neighbourhood"])

In [ ]:
df.sort_values(by=["price"], ascending=False)

In [ ]:
df.sort_values(by=["price"], ascending=False)[:5]

# Ejercicios (1)

In [ ]:
# Ejercicio: Por barrio (neighbourhood), la disponibilidad (availability_365) media para apartamentos/casas enteros (Entire home/apt),
# ordenados por disponibilidad media de mayor a menor
df.loc[df.room_type=="Entire home/apt"].groupby(["neighbourhood"]).availability_365.agg(["mean"]).sort_values(by=["mean"], ascending=False)

In [ ]:
# Ejercicio: Hosts con más de 10 opiniones en total

In [ ]:
g = df.groupby(["host_id"]).number_of_reviews.agg(["sum"])
g[g["sum"] > 10]

In [ ]:
# version 2
df.groupby(["host_id"]).number_of_reviews.agg(["sum"]).rename(columns={"sum": "suma"}).query("suma > 10")

In [ ]:
# Ejercicio: Top 5 propiedades por número de opiniones (number_of_reviews)
df.groupby(["name"]).number_of_reviews.agg(["sum"]).sort_values(by=["sum"], ascending=False).head(5)

# version alternativa
df.groupby(["name"]).number_of_reviews.agg(["sum"]).rename(columns={"sum":"suma"}).sort_values(by=["suma"],ascending=False)[:5]

# Funciones de fecha y cadena de caracteres

In [ ]:
df = pd.read_csv("http://data.insideairbnb.com/spain/comunidad-de-madrid/madrid/2023-03-15/visualisations/listings.csv", index_col="id", parse_dates=["last_review"])
df.last_review

In [ ]:
df.last_review.dt.day_of_week

In [ ]:
from datetime import datetime

df["dias_para_navidad"] = (datetime.strptime("2023-12-25", "%Y-%m-%d").date() - df.last_review.dt.date).dt.days
df["dias_para_navidad"]

In [ ]:
df.name.str.replace("Metro", "Subte")

# Ejercicios (2)

In [ ]:
# Ejercicio: Crear un campo nuevo que me dice si un apartamento es "Bed and Breakfast"
df["es_bed_and_brekfast"] = df.name.str.lower().str.replace("&", "and").str.contains("bed and breakfast").sum()

# Combinación de datos

In [ ]:
df_bcn = pd.read_csv("http://data.insideairbnb.com/spain/catalonia/barcelona/2023-03-14/visualisations/listings.csv", index_col="id")
df_bcn

## Combinar filas

In [ ]:
pd.concat([df, df_bcn], ignore_index=True)

## Combinar columnas (juntas)

In [ ]:
df_mean_price = df.groupby("neighbourhood").price.mean().reset_index().rename(columns={"price": "mean_price"})
df_mean_price

In [ ]:
df2 = df.merge(df_mean_price, on="neighbourhood")

df2.loc[(df2.price <= df2.mean_price * 0.8)]

In [ ]:
# Ejercicio: Por barrio, listado de propiedades con cantidad de opiniones (reviews) mayor al promedio de opiniones de ese barrio

df_mean_reviews =  df.groupby("neighbourhood").number_of_reviews.mean().reset_index().rename(columns={"number_of_reviews":"mean_number_of_reviews"})

df3 = df.merge(df_mean_reviews, on="neighbourhood")
df3.loc[(df3.number_of_reviews > df3.mean_number_of_reviews)]

# Visualización de datos

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [20, 8]

In [ ]:
df.neighbourhood_group.hist()

In [ ]:
df.hist(by="neighbourhood_group", column="neighbourhood")

In [ ]:
df.loc[df.price < 1000].price.hist(bins=100)

In [ ]:
df.loc[df.price < 200].boxplot(column="price")

In [ ]:
df.loc[df.price < 200].boxplot(by="neighbourhood_group", column="price")

In [ ]:
df.loc[df.room_type == "Hotel room"].boxplot(by="neighbourhood_group", column="price")

In [ ]:
df[df.price < 200].plot.scatter("availability_365", "price", c=df[df.price < 200].neighbourhood_group.map({
    'Chamartín': "red", 'Latina': "blue", 'Centro': "green", 'Salamanca': "orange", 'San Blas - Canillejas': "yellow", 'Ciudad Lineal': "orange", 'Hortaleza': "yellow" 
}).fillna("black"))



In [ ]:
df.neighbourhood_group.unique()

# Ejercicios (3)

In [ ]:
# Ejercicio: Histograma de la distribución de la cantidad mínima de noches (minimun_nights)
df.loc[df.minimum_nights < 90].minimum_nights.hist(bins=100)

In [ ]:
# Ejercicio: cantidad mínima de noches (minimum_nights) en promedio que me piden por tipo de habitación (room_type)
df.loc[df.minimum_nights < 10].boxplot(column="minimum_nights", by="room_type")

In [ ]:
# Ejercicio: Hacer un mapa de las propiedades, coloreando todo en azul, menos Carabanchel que va en rojo y Retiro en verde

plt.rcParams['figure.figsize'] = [10, 10]

df[df.price < 200].plot.scatter("longitude", "latitude", c=df[df.price < 200].neighbourhood_group.map({
    'Carabanchel': "red", 'Retiro': "green"}).fillna("blue"))